## Real World Production Data Manipulation
Disclaimer: For a privacy porpouse, this dataset have been generated for shocasing the code. The original is confidencial.

This dataset saves in an specific turn and specific machine the quantity of items that have been produced. It acumulates the amount until the turn finished.
We want to know the total amount of items produced in every turn (the maximum number of items produced)

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import itertools
import os

In [2]:
raw_data = pd.read_csv('../input/real-world-factory-production/produced.csv')

In [3]:
raw_data.dtypes

Unnamed: 0          int64
turn                int64
machineId           int64
date                int64
quantityProduced    int64
dtype: object

In [8]:
raw_data.drop('Unnamed: 0', axis=1, inplace=True)

### Unique ID
Getting an unique id for each production record

In [11]:
raw_data['turn_str'] = raw_data['turn'].apply(str)
raw_data['machineId_str'] = raw_data['machineId'].apply(str)
raw_data['date_str'] = raw_data['date'].apply(str)

raw_data['id'] = raw_data[['turn_str', 'machineId_str','date_str']].agg('-'.join, axis=1)
raw_data.drop(['turn_str','machineId_str','date_str'], axis=1, inplace=True)

raw_data.head()

,turn,machineId,date,quantityProduced,id
0,1,51,20210505,28,1-51-20210505
1,2,51,20210505,53,2-51-20210505
2,3,51,20210505,82,3-51-20210505
3,4,51,20210505,106,4-51-20210505
4,5,51,20210505,109,5-51-20210505


In [12]:
data = raw_data.copy()
indexes = []

In [24]:
def findIndexes(totals, column,turno, linea, fecha):
    if column == 'quantityProduced':
        for i in totals:
            inxs = data.index[(data.machineId == int(linea)) &
                    (data.date == int(fecha)) &
                    (data.quantityProduced == i)].values
            indexes.append(inxs)

def getTotalProduced(id, column):
    turno, linea, fecha = id.split('-')
    totals = data[(data.machineId == int(linea)) & (data.date == int(fecha))].groupby(['turn'])[column].max();
    
    findIndexes(totals, column, turno, linea, fecha)

    totals_diff = totals.diff()
    
    if int(turno) == 1:
        return(totals[1])
    else:
        return(totals_diff[int(turno)])
    return i[0]

### Loop over rows

In [25]:
name = 'ProducedPerTurn'
data[name] = data['id'].apply(lambda x: getTotalProduced(x, 'quantityProduced'))

In [28]:
pd.options.display.float_format = '{:,.0f}'.format
data.drop(['id', 'quantityProduced'], axis=1, inplace=True)
data.head()

,turn,machineId,date,ProducedPerTurn
0,1,51,20210505,28
1,2,51,20210505,25
2,3,51,20210505,29
3,4,51,20210505,24
4,5,51,20210505,3


## Done! We have reached the desired insight of this dataset!